In [1]:
!pip install easyocr
!pip install opencv-python-headless==4.1.2.30
#!pip install Pillow
#!pip install ftfy

     |████████████████████████████████| 70.8 MB 12 kB/s 
     |████████████████████████████████| 47.6 MB 1.6 MB/s 
     |████████████████████████████████| 21.8 MB 5.4 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.4.60
    Uninstalling opencv-python-headless-4.5.4.60:
      Successfully uninstalled opencv-python-headless-4.5.4.60


In [2]:
import json
import cv2
import re
import numpy as np
import easyocr
reader = easyocr.Reader(['en'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [48]:
def pan_read_data(text):
  name = None
  fname = None
  dob = None
  pan = None
  text1 = []
  lines = text.split(',')
  for lin in lines:
    s = lin.strip()
    s = lin.replace('\n','')
    s = s.rstrip()
    s = s.lstrip()
    text1.append(s)
  text1 = list(filter(None, text1))
  try:
    dob = re.search(r'\d{2}.\d{2}.\d{4}',text).group()
  except:
    pass
  
  try:
    pan = re.search(r'[a-zA-Z]{5}\d{4}[a-zA-Z]',text).group()
  except:
    lineno = 0
    for wordline in text1:
      xx = wordline.split(' ')
      if ([w for w in xx if re.search('(Pormanam|Number|umber|Account|ccount|count|Permanent|ermanent|manent|wumm)$', w)]):
          text1 = list(text1)
          lineno = text1.index(wordline)            
          pan = text1[lineno+1:][0]

  lineno = 0
  for wordline in text1:
    xx = wordline.split(' ')
    if ([w for w in xx if re.search('(Name|GOW|GOVT|GOVERNMENT|OVERNMENT|VERNMENT|INDIA|NDIA)$', w)]):
      if ([w for w in xx if re.search('(Father|FATHER|Fath|ather)$', w)]):
        text1 = list(text1)
        lineno = text1.index(wordline)
        fname = text1[lineno+1:][0]
        break
      text1 = list(text1)
      lineno = text1.index(wordline)
      name = text1[lineno+1:][0]
      fname = text1[lineno+1:][1]
                
  try:
# Cleaning first names
      name = name.rstrip()
      name = name.lstrip()
      name = name.replace("8", "B")
      name = name.replace("0", "D")
      name = name.replace("6", "G")
      name = name.replace("1", "I")
      name = re.sub('[^a-zA-Z] +', ' ', name)
# Cleaning Father's name
      fname = fname.rstrip()
      fname = fname.lstrip()
      fname = fname.replace("8", "S")
      fname = fname.replace("0", "O")
      fname = fname.replace("6", "G")
      fname = fname.replace("1", "I")
      fname = fname.replace("\"", "A")
      fname = re.sub('[^a-zA-Z] +', ' ', fname)

  except:
          pass
  data = {}
  data['Name'] = name
  data['Father Name'] = fname
  data['Date of Birth'] = dob
  data['PAN'] = pan
  data['ID Type'] = "PAN"
  return data


In [139]:
def adhaar_read_data(text):
    name = None
    dob = None
    adh = None
    sex = None
    text1 = []
    lines = text.split(',')
    for lin in lines:
      s = lin.strip()
      s = lin.replace('\n','')
      s = s.rstrip()
      s = s.lstrip()
      text1.append(s)

    if 'female' in text.lower():
        sex = "FEMALE"
    else:
        sex = "MALE"

    # extractin DOB
    try:
      dob = re.search(r'\d{2}.\d{2}.\d{4}',text).group()
    except:
      pass
    try:
      adh = re.search(r'\d{4}\s\d{4}\s\d{4}',text).group()
    except:
      pass

    lineno = 0
    for wordline in text1:
      xx = wordline.split(' ')
      if ([w for w in xx if re.search('(Govt|Government|India|Government of India|lndia|Indla|GOW|GOVT|GOVERNMENT|OVERNMENT|VERNMENT|INDIA|NDIA)$', w)]):
          text1 = list(text1)
          lineno = text1.index(wordline) 
          name = text1[lineno+2]           
    
    try:   

      # Cleaning first names
      name = name.rstrip()
      name = name.lstrip()
      name = name.replace("8", "B")
      name = name.replace("0", "D")
      name = name.replace("6", "G")
      name = name.replace("1", "I")
      name = re.sub('[^a-zA-Z] +', ' ', name)

    except:
      pass

    data = {}
    data['Name'] = name
    data['Date of Birth'] = dob
    data['Adhaar Number'] = adh
    data['Sex'] = sex
    data['ID Type'] = "Adhaar"
    return data

def detect_text_blocks(img_path):
    detection_result = reader.detect(img_path,
                                 width_ths=0.7, 
                                 mag_ratio=1.5
                                 )
    text_coordinates = detection_result[0][0]
    return text_coordinates



In [5]:
def extractinfo(imgpath):
  text_coordinates = detect_text_blocks(imgpath)
  recognition_results = reader.recognize(imgpath,horizontal_list=text_coordinates,free_list=[])
  text = [txt[1] for txt in recognition_results]
  text = ",".join(text)
  data = None
  if "income" in text.lower() or "tax" in text.lower() or "department" in text.lower():
      data = pan_read_data(text)
  
  elif "male" in text.lower():
      data = adhaar_read_data(text)
  return data


In [145]:
imgpath = "/content/aadhar1.jpg"
extractinfo(imgpath)

{'Adhaar Number': '7389 4269 3567',
 'Date of Birth': '15/02/1990',
 'ID Type': 'Adhaar',
 'Name': 'Abdul Khalek Mondal',
 'Sex': 'MALE'}

In [146]:
imgpath = "/content/Pansample1.jpg"
extractinfo(imgpath)

{'Date of Birth': '16/07/1986',
 'Father Name': 'DURAISAMY',
 'ID Type': 'PAN',
 'Name': 'D MANIKANDAN',
 'PAN': 'BNZPM2SOIF'}